In [ ]:
import pandas as pd
import numpy as np

from sklearn import preprocessing
import matplotlib.pyplot as plt 
plt.rc("font", size=14)

import seaborn as sns
sns.set(style="white") #white background style for seaborn plots
sns.set(style="whitegrid", color_codes=True)
import matplotlib.pyplot as plt


import warnings
warnings.simplefilter(action='ignore')

#classification model
from sklearn import linear_model
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import Perceptron
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC, LinearSVC
from sklearn.naive_bayes import GaussianNB




# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
test_data=pd.read_csv("/kaggle/input/titanic/test.csv")
train_data=pd.read_csv("/kaggle/input/titanic/train.csv")
gender_submission=pd.read_csv("/kaggle/input/titanic/gender_submission.csv")

#print(test_data)
train_data.head()
#print(gender_submission)

In [ ]:
print('The number of samples into the train data is {}.'.format(train_data.shape[0]))

In [ ]:
train_data.isnull().sum()

In [ ]:
#missing percentage of age info
print("% of missing age")
print(round(train_data['Age'].isnull().sum()/train_data.shape[0]*100,2))


#visualizing distribution of age
ax = train_data["Age"].hist(bins=15, density=True, stacked=True, color='teal', alpha=0.6)
train_data["Age"].plot(kind='density', color='blue')
ax.set(xlabel='Age')
plt.xlim(0,85)
plt.show()


#mean age
mean_age=round(train_data['Age'].sum()/(train_data.shape[0]-train_data['Age'].isnull().sum()),2)
median_age=train_data['Age'].median(skipna=True)
print("mean age ",mean_age)
print("median age ",median_age)

In [ ]:
#missing percentage of cabin info
print("% of missing cabin info")
print(round(train_data['Cabin'].isnull().sum()/train_data.shape[0]*100,2))
print('we should ignore this feature for our model')


In [ ]:
#embarking missing values
print("% of missing cabin info")
print(round(train_data['Embarked'].isnull().sum()/train_data.shape[0]*100,2))

#we'll it to most boarded port
print(train_data['Embarked'].value_counts())
#here most boarded port is S


In [ ]:
#filling the missing values for age and Embarked and dropping the feature cabin

#train_data1 = train_data.copy()
#train_data1["Age"].fillna(train_data["Age"].median(skipna=True), inplace=True)
#train_data1["Embarked"].fillna(train_data['Embarked'].value_counts().idxmax(), inplace=True)
#train_data1.drop('Cabin', axis=1, inplace=True)
#train_data1.head()

In [ ]:
train=train_data
test=test_data
full_data = [train, test]

# Some features of my own that I have added in
# Gives the length of the name
train['Name_length'] = train['Name'].apply(len)
test['Name_length'] = test['Name'].apply(len)
# Feature that tells whether a passenger had a cabin on the Titanic
train['Has_Cabin'] = train["Cabin"].apply(lambda x: 0 if type(x) == float else 1)
test['Has_Cabin'] = test["Cabin"].apply(lambda x: 0 if type(x) == float else 1)

# Feature engineering steps taken from Sina
# Create new feature FamilySize as a combination of SibSp and Parch
for dataset in full_data:
    dataset['FamilySize'] = dataset['SibSp'] + dataset['Parch'] + 1
# Create new feature IsAlone from FamilySize
for dataset in full_data:
    dataset['IsAlone'] = 0
    dataset.loc[dataset['FamilySize'] == 1, 'IsAlone'] = 1
# Remove all NULLS in the Embarked column
for dataset in full_data:
    dataset['Embarked'] = dataset['Embarked'].fillna('S')
# Remove all NULLS in the Fare column and create a new feature CategoricalFare
for dataset in full_data:
    dataset['Fare'] = dataset['Fare'].fillna(train['Fare'].median())
train['CategoricalFare'] = pd.qcut(train['Fare'], 4)
# Create a New feature CategoricalAge
for dataset in full_data:
    age_avg = dataset['Age'].mean()
    age_std = dataset['Age'].std()
    age_null_count = dataset['Age'].isnull().sum()
    age_null_random_list = np.random.randint(age_avg - age_std, age_avg + age_std, size=age_null_count)
    dataset['Age'][np.isnan(dataset['Age'])] = age_null_random_list
    dataset['Age'] = dataset['Age'].astype(int)
train['CategoricalAge'] = pd.cut(train['Age'], 5)

for dataset in full_data:
    # Mapping Sex
    dataset['Sex'] = dataset['Sex'].map( {'female': 0, 'male': 1} ).astype(int)
    
    # Mapping Embarked
    dataset['Embarked'] = dataset['Embarked'].map( {'S': 0, 'C': 1, 'Q': 2} ).astype(int)
    
    # Mapping Fare
    dataset.loc[ dataset['Fare'] <= 7.91, 'Fare'] 						        = 0
    dataset.loc[(dataset['Fare'] > 7.91) & (dataset['Fare'] <= 14.454), 'Fare'] = 1
    dataset.loc[(dataset['Fare'] > 14.454) & (dataset['Fare'] <= 31), 'Fare']   = 2
    dataset.loc[ dataset['Fare'] > 31, 'Fare'] 							        = 3
    dataset['Fare'] = dataset['Fare'].astype(int)
    
    # Mapping Age
    dataset.loc[ dataset['Age'] <= 16, 'Age'] 					       = 0
    dataset.loc[(dataset['Age'] > 16) & (dataset['Age'] <= 32), 'Age'] = 1
    dataset.loc[(dataset['Age'] > 32) & (dataset['Age'] <= 48), 'Age'] = 2
    dataset.loc[(dataset['Age'] > 48) & (dataset['Age'] <= 64), 'Age'] = 3
    dataset.loc[ dataset['Age'] > 64, 'Age'] = 4 ;

In [ ]:
#checking the data
dataset.isnull().sum()

In [ ]:
drop_elements = ['PassengerId', 'Name', 'Ticket', 'Cabin', 'SibSp', 'Name_length', 'Has_Cabin','Parch']
train = train.drop(drop_elements, axis = 1)
train = train.drop(['CategoricalAge', 'CategoricalFare'], axis = 1)
test  = test.drop(drop_elements, axis = 1)

In [ ]:
train.head()
#test.head()

In [ ]:
svc_params = {
    'kernel' : 'linear',
    'C' : 0.025
    }

In [ ]:
y_train = train['Survived'].ravel()
train = train.drop(['Survived'], axis=1)
x_train = train.values # Creates an array of the train data
x_test = test.values # Creats an array of the test data


In [ ]:
#print(y_train)
#print(x_train)
#print(x_test)

In [ ]:
#Linear SVM
linear_svc = LinearSVC()
linear_svc.fit(x_train, y_train)

Y_pred_svm = linear_svc.predict(x_test)

acc_linear_svc = round(linear_svc.score(x_train, y_train) * 100, 2)
print("linear SVM: ",acc_linear_svc)

In [ ]:
#Gaussain Naive Bayes
gaussian = GaussianNB()
gaussian.fit(x_train, y_train)  
Y_pred_gauss = gaussian.predict(x_test)  
acc_gaussian = round(gaussian.score(x_train, y_train) * 100, 2)
print("Gaussain Naive Bayes: ",acc_gaussian)

In [ ]:
#logistic Regression
logreg = LogisticRegression()
logreg.fit(x_train, y_train)

Y_pred_log = logreg.predict(x_test)

acc_log = round(logreg.score(x_train, y_train) * 100, 2)
print("Logistic Regression: ", acc_log)

In [ ]:
#K-Nearest Neighbours
knn = KNeighborsClassifier(n_neighbors = 3)
knn.fit(x_train, y_train)  
Y_pred_knn = knn.predict(x_test)  
acc_knn = round(knn.score(x_train, y_train) * 100, 2)
print("knn: ", acc_knn)

In [ ]:
#cross validation
from sklearn.model_selection import cross_val_score
kn = KNeighborsClassifier(n_neighbors = 3)
scores = cross_val_score(kn, x_train, y_train, cv=10, scoring = "accuracy")
print("Scores:", scores)
print("Mean:", scores.mean())
print("Standard Deviation:", scores.std())

In [ ]:
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
predictions = cross_val_predict(knn, x_train, y_train, cv=10)
confusion_matrix(y_train, predictions)

In [ ]:
from mlxtend.plotting import plot_decision_regions
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from matplotlib.colors import ListedColormap

X_train_reduced = PCA(n_components = 2).fit_transform(x_train)
t = np.array(y_train)
t = t.astype(np.integer)
kf=KNeighborsClassifier(n_neighbors = 3)
kf.fit(X_train_reduced,t)
plt.figure(figsize = [15,10])
plot_decision_regions(X_train_reduced, t, clf = kf, hide_spines = False, colors = 'purple,limegreen',
                      markers = ['^','v'])
plt.title('K-Nearst Neighbours')

In [ ]:
test_data.head()

In [ ]:
clf_knn =KNeighborsClassifier(n_neighbors = 3)
clf_knn.fit(x_train, y_train)  
test_df = test_data.drop(['Name','Has_Cabin','Ticket','Name_length','Cabin','SibSp','Parch'],axis = 1)
test_df.head()
#Pclass	Sex	Age	Fare	Embarked	FamilySize	IsAlone



In [ ]:
nptest = np.array(test_df)
pred = clf_knn.predict(x_test)
predictions = pd.DataFrame(pred, index = test_df.index, columns = ['Survived'])
df= predictions['Survived']
final=test_df['PassengerId']
final=pd.concat([final,df],axis=1,join='inner')
final.to_csv('predictions_knn.csv',index=False)
output=pd.read_csv("predictions_knn.csv")

final


